# Landsattrend data preparation script for clowder extractor

In [1]:
import ee#, eemont
#ee.Authenticate()
ee.Initialize()

import geemap
from importlib import reload  
import geopandas as gpd
import pandas as pd

from GEE_HotSpot.modules import high_level_functions
from GEE_HotSpot.modules import utils_Landsat_SR as utils_LS
from GEE_HotSpot.modules import ms_indices as indices
from GEE_HotSpot.modules import configs, utils_string

from utils.utils_processing import *

import numpy as np
from pathlib import Path
import os, shutil
import tqdm
import time

In [2]:
def run_preprocess(config_trend, crs='EPSG:32656', prefix='trendimage_Z056-Kolyma', tasking=True, mask_non_water=True):
    trend = high_level_functions.runTCTrend(config_trend)
    data = trend['data']

    # setup data
    dem = create_dem_data()
    data_export = data.addBands(dem).toFloat().select(data_cols)
    if mask_non_water:
        # new version set to 90 m
        water_mask = get_water_mask(dilation_size=90)
        data_export = data_export.updateMask(water_mask)
    if tasking:
        # Export
        task = ee.batch.Export.image.toDrive(
            image=data_export,
            description=prefix,
            folder=OUTFOLDER,
            fileNamePrefix=prefix,
            crs=crs,
            region=geom,
            scale=30,
            maxPixels=1e12)
        task.start()
        return None
    else:
        return data_export

In [3]:
data_cols = ['TCB_slope',
             'TCB_offset',
             'TCG_slope', 
             'TCG_offset', 
             'TCW_slope',
             'TCW_offset',
             'NDVI_slope',
             'NDVI_offset',
             'NDMI_slope',
             'NDMI_offset',
             'NDWI_slope',
             'NDWI_offset', 
             'elevation', 
             'slope'] 

### Run  Batch Process

#### Project Properties - Change Settings Here 

In [4]:
# PROPERTIES
# SET METADATA AND EXPORT PARAMETERS
MAXCLOUD = 70
STARTYEAR = 2000
ENDYEAR = 2020
STARTMONTH = 7
ENDMONTH = 8
SCALE = 30
OUTFOLDER = 'PDG_Trend'

# SET SPATIAL PARAMETERS
UTM_ZONE = 11
LATITUDE_MIN = 60
LATITUDE_MAX = 80

# SPATIAL TILING SETTINGS (can be kept as is!)
Y_SIZE = 1
X_SIZE = 3
X_OVERLAP = 0.1
X_SIZE_BUFFERED = X_SIZE + (2*X_OVERLAP)


In [5]:
# image metadata Filters
config_trend = {
  'date_filter_yr' : ee.Filter.calendarRange(STARTYEAR, ENDYEAR, 'year'),
  'date_filter_mth' : ee.Filter.calendarRange(STARTMONTH, ENDMONTH, 'month'),
  'meta_filter_cld' : ee.Filter.lt('CLOUD_COVER', MAXCLOUD),
  'select_bands_visible' : ["B1", "B2","B3","B4"],
  'select_indices' : ["TCB", "TCG", "TCW", "NDVI", "NDMI", "NDWI"],
  'select_TCtrend_bands' : ["TCB_slope", "TCG_slope", "TCW_slope"],
  'geom' : None
}
#------ RUN FULL PROCESS FOR ALL REGIONS IN LOOP ------------------------------
#Map.addLayer(imageCollection, {}, 'TCVIS')

In [13]:
#longitudes = range(90, 96, X_SIZE)
longitudes = get_lon_from_utmzone(UTM_ZONE, X_SIZE)
print(list(longitudes))

latitudes = range(LATITUDE_MIN, LATITUDE_MAX, Y_SIZE)
print(list(latitudes))

[-120, -117]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]


#### Run Processing 

In [14]:
RUN_PROCESS = 1

In [15]:
for X_MIN in longitudes:
    for Y_MIN in latitudes:
        geom = ee.Geometry.Rectangle(coords=[X_MIN-X_OVERLAP, Y_MIN, X_MIN+X_SIZE_BUFFERED, Y_MIN+Y_SIZE], proj=ee.Projection('EPSG:4326'))
        config_trend['geom'] = geom
        utm = get_utmzone_from_lon(X_MIN)
        
        crs = crs_from_utmzone(utm)
        epsg = epsg_from_utmzone(utm)
        prefix = make_fileprefix(epsg=epsg, period_start=STARTYEAR, period_end=ENDYEAR, lon=X_MIN, lat=Y_MIN)
        print(prefix)
        if RUN_PROCESS:
            run_preprocess(config_trend, crs=crs, prefix=prefix, mask_non_water=True)

trendimage_2000-2020_32611_-120_60
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32611_-120_61
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32611_-120_62
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32611_-120_63
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32611_-120_64
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32611_-120_65
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32611_-120_66
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32611_-120_67
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32611_-120_68
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32611_-120_69
['B1', 'B2'